In [92]:
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import torchvision
import torch
import os
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, auc
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import pickle
from sklearn.metrics import accuracy_score, f1_score
import torch.utils.data as data_utils
from collections import defaultdict
import gc

In [93]:
def load_and_transform_cifar100(label_noise=0.0):
    training_data = datasets.CIFAR100(
        root="data",
        train=True,
        download=True,
    )

    test_data = datasets.CIFAR100(
        root="data",
        train=False,
        download=True,
    )
    
    if label_noise > 0:
        size = int(label_noise * len(training_data))
        noise_idx = set(np.random.choice(len(training_data), size, replace=False))
        print('injecting noise ...%s' %len(noise_idx))
        targets = []
        for idx, t in enumerate(training_data.targets):
            if idx in noise_idx:
                targets.append(np.random.randint(100))
            else:
                targets.append(t)
                
        training_data.targets = targets
        
    image_size = (128, 128)
    resnet_mean = [0.485, 0.456, 0.406]
    resnet_std = [0.229, 0.224, 0.225]

    #Creating a Transformation Object
    data_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size=image_size),
        torchvision.transforms.ToTensor(), #Converting to tensor
        torchvision.transforms.Normalize(mean = resnet_mean,
                                        std = resnet_std) 
    ])
    training_data.transform = data_transform
    test_data.transform = data_transform
    return training_data, test_data
    

In [ ]:
#creates original cifar coarse to fine and back dictionaries
 
class Dictlist(dict):
    def __setitem__(self, key, value):
        try:
            self[key]
        except KeyError:
            super(Dictlist, self).__setitem__(key, [])
        self[key].append(value)
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return dic
 
x=unpickle('data/cifar-100-python/test')
d=Dictlist()
 
for i in range(0,len(x[b'coarse_labels'])):
    d[x[b'coarse_labels'][i]]=x[ b'fine_labels'][i]
coarse_to_fine=dict(d)
for i in coarse_to_fine.keys():
    coarse_to_fine[i]=list(dict.fromkeys(coarse_to_fine[i]))

fine_to_coarse = dict()
for k,v in coarse_to_fine.items():
    for each in v:
        fine_to_coarse[each] = k

In [94]:
def filter_data(training_data, test_data, filter_by_coarse=False, 
                coarse_label=None, coarse_to_fine=None, valid_size=0.1, batch_size=64):
    """Filters datasets by coarse label if needed and split to train and validation subsets"""
    if filter_by_coarse:
        labels = coarse_to_fine[coarse_label]
        indices = [idx for idx,v in enumerate(training_data.targets) if v in labels]
        indices_test = [idx for idx,v in enumerate(test_data.targets) if v in labels]     
    else:
        indices = [i for i in range(len(training_data.targets))]
        indices_test = [i for i in range(len(test_data.targets))]
    validate_len = int(len(indices)*valid_size)
    validate_filter_indices = random.sample(range(0,len(indices)), validate_len)
    indices_val = [indices[i] for i in validate_filter_indices]  
    indices_train = [v for i,v in enumerate(indices) if i not in validate_filter_indices]    
        
    train = data_utils.Subset(training_data, indices_train)
    validate = data_utils.Subset(training_data, indices_val)
    test = data_utils.Subset(test_data, indices_test)
    
    train_dl = DataLoader(train,  batch_size=batch_size, shuffle=True)
    validate_dl = DataLoader(validate, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test,  batch_size=batch_size, shuffle=True)
    return train_dl, validate_dl, test_dl

In [95]:
def create_label_dictionaries(cluster_list):
    """Creates dictionaries from coarse to fine, from fine to coarse and concatenated list 
    of fine clusters
    
    Parameters:
    cluster_list: list
        A list where the index of the element represents the fine label and the value represents
        the coarse label(cluster)    
    """
    fine_to_coarse = {i:v for i,v in enumerate(cluster_list)}
    coarse_to_fine = defaultdict(list)
    for k, v in fine_to_coarse.items():
        coarse_to_fine[v].append(k)
    return fine_to_coarse, coarse_to_fine




# #creates original cifar coarse to fine and back dictionaries

# class Dictlist(dict):
#     def __setitem__(self, key, value):
#         try:
#             self[key]
#         except KeyError:
#             super(Dictlist, self).__setitem__(key, [])
#         self[key].append(value)

# def unpickle(file):
#     import pickle
#     with open(file, 'rb') as fo:
#         dic = pickle.load(fo, encoding='bytes')
#     return dic


# x=unpickle('data/cifar-100-python/test')
# d=Dictlist()

# for i in range(0,len(x[b'coarse_labels'])):
#     d[x[b'coarse_labels'][i]]=x[ b'fine_labels'][i]
# coarse_to_fine=dict(d)
# for i in coarse_to_fine.keys():
#     coarse_to_fine[i]=list(dict.fromkeys(coarse_to_fine[i]))
    
# fine_to_coarse = dict()
# for k,v in coarse_to_fine.items():
#     for each in v:
#         fine_to_coarse[each] = k
    

In [96]:
class Resnext50(torch.nn.Module):
    def __init__(self, n_classes, name):
        super().__init__()
        self.name = name
        resnet = torchvision.models.resnext50_32x4d(pretrained=True, progress=True)
        resnet.fc = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.soft = torch.nn.Softmax()

    def forward(self, x):
        return self.soft(self.base_model(x))   
    

In [97]:
def train_loop(dataloader, model, loss_fn, optimizer, mode="flat", lookup=[], fine_to_coarse={}, batch_size=64):
    size = len(dataloader.dataset)
    model.train()
    train_loss = 0
    counter = 0
    with tqdm(dataloader, unit="batch") as tepoch:
        counter += 1
        for X_cpu, y_cpu in tepoch:
            if mode == "fine":
                y_cpu = torch.tensor([lookup.index(i) for i in y_cpu])
            elif mode == "coarse":
                y_cpu = torch.tensor([fine_to_coarse[int(target)] for target in y_cpu])
            X, y = X_cpu.to(device), y_cpu.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = loss_fn(outputs, y.type(torch.long))
            loss.backward() 
            optimizer.step() 
            train_loss += loss.item() 
            avg_loss = train_loss / (counter * batch_size)
            tepoch.set_postfix(loss=avg_loss)
    return avg_loss
    
    
def eval_model(dataloader, model, mode="flat", lookup=[], fine_to_coarse={}, return_probab=False):
    model.eval()
    predicts = []
    targets = []
    for counter, (images, labels) in enumerate(dataloader):
        if mode == "fine":
            labels = [lookup.index(i) for i in labels]
        elif mode == "coarse":
            labels = [fine_to_coarse[int(i)] for i in labels]
        imageGPU = images.to(device)
        
        outputs = torch.Tensor.cpu(model(imageGPU))
        predicts.append(outputs.detach().numpy())
        targets.append(labels)
        
    predicts = np.vstack(predicts)
    targets = np.hstack(targets)
    if return_probab:
        return predicts, targets
    else:
        predicts = np.argmax(predicts, axis=1)
        return accuracy_score(targets, predicts)
    


In [98]:
def train_and_validate(epochs, model, loss_fn, optimizer, train_dl, val_dl, test_dl, 
                       mode="flat", fine_to_coarse={}, coarse_label=None, lookup=[], model_path = None):
    if model_path is None:
        model_path = '/models/%s.pth' %model.name
    min_loss= 1000
    max_acc = 0
    for t in range(epochs):
        if mode == "fine":
            print(f"Coarse label {coarse_label} Epoch {t+1}\n-------------------------------")
        else:
            print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        val_acc = eval_model(val_dl, model, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        if train_loss < min_loss:
            max_acc = val_acc
            min_loss = train_loss
            torch.save(model, model_path)
        elif train_loss == min_loss and val_acc>max_acc:
            max_acc = val_acc
            min_loss = train_loss
            torch.save(model, model_path)
        print("validation acc %s and loss %s" %(val_acc, train_loss))
        test_acc = eval_model(test_dl, model, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        print("done epoch %s : test_acc %s" %(t, test_acc))
    print("Done! Saved model with validation accuracy %s and loss %s" %(max_acc, min_loss))
    return (max_acc, test_acc)

In [102]:
#load clusters data
tasksim_clusters = pickle.load(open('../../hierarchical/notebooks/real-data/cifar-paper-experiments/cifar_tasksim_clusters_1.pkl', 'rb'))
tasksim_clusters

[array([11,  6,  7, 15,  3,  2,  5, 17,  2, 16, 23,  7, 12, 22,  6,  4, 18,
        12,  6, 13,  0,  3, 18,  8,  6,  0,  6,  3, 18,  4, 20,  4, 21,  1,
        14,  7,  3, 12, 15, 18, 23,  2,  3, 14,  6,  6,  7,  1, 22,  8, 15,
         4, 19, 11,  5,  3, 13, 23,  9,  1,  1, 23,  5, 15, 15, 15, 15, 21,
        12, 12,  5,  8, 20, 10, 15, 15, 12,  6,  6,  6, 15,  9,  5, 23,  0,
        12, 18,  0, 15,  2, 12, 21, 24, 21,  0, 20,  1, 14,  7,  6])]

In [103]:
acorn = 1234
torch.manual_seed(acorn)
np.random.seed(acorn)

torch.cuda.is_available()

if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
    
device = torch.device(dev)  

batch_size = 64
valid_size = 0.1

In [ ]:
#Prepare data for a coarse classifier training

fine_to_coarse, coarse_to_fine = create_label_dictionaries(tasksim_clusters[0])
number_of_clusters = len(coarse_to_fine.keys())
training_data, test_data=load_and_transform_cifar100()
train_dl, validate_dl, test_dl = filter_data(training_data, test_data, valid_size=valid_size, batch_size=batch_size)

#coarse classifier training

learning_rate = 1e-4
epochs = 15
model = Resnext50(number_of_clusters, name="hierarchical_coarse_model_Resnext50_25components")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.to(device)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

coarse_max_acc, coarse_test_acc =train_and_validate(epochs, model, loss_fn, optimizer, train_dl, validate_dl, test_dl, 
                                                    mode="coarse", fine_to_coarse=fine_to_coarse,
                                                    model_path = 'models/hierarchical/%s.pth' %model.name )
print("coarse accuracy %s" %coarse_test_acc)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/704 [00:00<?, ?batch/s]

Epoch 1
-------------------------------


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 704/704 [16:33<00:00,  1.41s/batch, loss=29.6]


validation acc 0.68 and loss 29.57446577399969


...

/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 704/704 [16:29<00:00,  1.41s/batch, loss=27.3]


validation acc 0.749 and loss 27.266039077192545


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
  0%|          | 0/704 [00:00<?, ?batch/s]

done epoch 8 : test_acc 0.7502
Epoch 10
-------------------------------


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
 28%|██▊       | 197/704 [04:37<12:17,  1.46s/batch, loss=7.59]

In [67]:
#fine clusters training

learning_rate = 1e-4
epochs = 10
accuracies = {}


for coarse_label in range(0, number_of_clusters):
        if len(coarse_to_fine[coarse_label])>1:
            train_dl, val_dl, test_dl =  filter_data(training_data, test_data, filter_by_coarse=True, 
                    coarse_label=coarse_label, coarse_to_fine=coarse_to_fine, valid_size=0.1, batch_size=batch_size)
            # Initialize the model
            model = Resnext50(len(coarse_to_fine[coarse_label]), name="hierarchical_fine_model_coarse_label_%s_25componentss" %coarse_label)
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            model.to(device)
            loss_fn = torch.nn.CrossEntropyLoss().to(device)
            max_acc, test_acc = train_and_validate(epochs, model, loss_fn, optimizer, train_dl, val_dl, test_dl, mode="fine", 
                                                   coarse_label=coarse_label, lookup=coarse_to_fine[coarse_label], 
                                                   model_path = 'models/hierarchical/%s.pth' %model.name )
            accuracies[coarse_label] = (max_acc, test_acc)
    print(accuracies)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/29 [00:00<?, ?batch/s]

Coarse label 0 Epoch 1
-------------------------------


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 29/29 [00:39<00:00,  1.38s/batch, loss=0.45] 


current acc 0.93 and loss 0.45014575589448214


...

done epoch 8 : test_acc 0.9333333333333333
Coarse label 27 Epoch 10
-------------------------------


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 22/22 [00:31<00:00,  1.42s/batch, loss=0.201] 


current acc 0.9066666666666666 and loss 0.20106425043195486


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


done epoch 9 : test_acc 0.9266666666666666
Done! Saved model with validation accuracy 0.9266666666666666 and loss 0.19184883311390877
{0: (0.935, 0.925), 1: (0.84, 0.8433333333333334), 2: (0.98, 0.9266666666666666), 3: (0.96, 0.94), 4: (0.98, 0.9866666666666667), 7: (0.964, 0.972), 8: (0.8775, 0.8675), 9: (0.932, 0.864), 10: (0.96, 0.975), 11: (0.9022222222222223, 0.8677777777777778), 12: (0.8733333333333333, 0.8916666666666667), 13: (0.92, 0.9433333333333334), 14: (0.95, 0.9483333333333334), 15: (0.96, 0.98), 16: (0.715, 0.6975), 18: (0.99, 0.935), 21: (0.812, 0.822), 23: (0.96, 0.915), 25: (0.89, 0.8875), 27: (0.9266666666666666, 0.9266666666666666)}


In [87]:
accuracies

{0: (0.935, 0.925),
 1: (0.84, 0.8433333333333334),
 2: (0.98, 0.9266666666666666),
 3: (0.96, 0.94),
 4: (0.98, 0.9866666666666667),
 7: (0.964, 0.972),
 8: (0.8775, 0.8675),
 9: (0.932, 0.864),
 10: (0.96, 0.975),
 11: (0.9022222222222223, 0.8677777777777778),
 12: (0.8733333333333333, 0.8916666666666667),
 13: (0.92, 0.9433333333333334),
 14: (0.95, 0.9483333333333334),
 15: (0.96, 0.98),
 16: (0.715, 0.6975),
 18: (0.99, 0.935),
 21: (0.812, 0.822),
 23: (0.96, 0.915),
 25: (0.89, 0.8875),
 27: (0.9266666666666666, 0.9266666666666666)}

In [70]:
#Evaluation of the whole model

stacked_fine_labels = []
for i in range(0, number_of_clusters):
    stacked_fine_labels +=coarse_to_fine[i]
    
#loading models
model_coarse = torch.load('models/hierarchical/hierarchical_coarse_model_Resnext50_25components.pth')
fine_models = {}
for i in range(0,number_of_clusters):
        fine_models['fine_model_' + str(i)] = torch.load('models/hierarchical/hierarchical_fine_model_coarse_label_%s_25components.pth' %i)
    test_batch_dl = DataLoader(test_data,  batch_size=batch_size, shuffle=False)


In [71]:
coarse_predicts = []
targets = []
model_coarse.to(device)
model_coarse.eval()
for counter, (images, labels) in enumerate(test_batch_dl):
    imageGPU = images.to(device)
    outputs = torch.Tensor.cpu(model_coarse(imageGPU))
    coarse_predicts.append(outputs.detach().numpy())
    targets.append(labels)
targets = np.hstack(targets)
coarse_predicts

/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


[array([[1.17139776e-09, 7.03188285e-10, 6.90449475e-10, ...,
         3.85776466e-10, 1.81144180e-11, 5.59139846e-10],
        [1.23844181e-11, 1.41367751e-09, 4.15017543e-12, ...,
         7.81111564e-11, 3.57924302e-13, 1.90585464e-10],
        [2.54305760e-11, 1.25869075e-04, 7.38693495e-10, ...,
         3.81293530e-09, 9.57888716e-12, 5.39667377e-09],
        ...,
        [3.40451833e-10, 9.99833584e-01, 6.11129591e-10, ...,
         3.19833052e-05, 1.82941190e-10, 7.66280479e-08],
        [3.02159691e-07, 3.20970616e-06, 1.21742276e-04, ...,
         1.14892482e-05, 8.96137848e-08, 6.82397187e-02],
        [2.49562388e-13, 9.99996543e-01, 4.01275557e-13, ...,
         4.97494320e-12, 1.12528009e-13, 4.76969131e-10]], dtype=float32),
 array([[9.64096546e-01, 6.71050338e-06, 1.26513260e-04, ...,
         6.54924315e-06, 8.19526008e-07, 6.97018413e-06],
        [1.06768630e-06, 1.27861213e-07, 5.20408832e-07, ...,
         9.00701480e-07, 3.41285578e-09, 1.59551320e-07],
        [2

In [77]:
#reformat coarse prediction
coarse_prediction = np.vstack(coarse_predicts)

column_num = 0
for i in range(0, number_of_clusters):
    cluster_len = len(coarse_to_fine[i])
    for rep in range(cluster_len-1):
        coarse_prediction = np.insert(coarse_prediction, column_num, coarse_prediction[:,column_num], axis=1)
    column_num +=cluster_len

In [83]:
fine_prediction= []

for i in range(0,number_of_clusters):
    if len(coarse_to_fine[i]) == 1:
        fine_prediction.append(np.ones((len(coarse_prediction),1)))
    else:
        predicts = []
        print("Evaluating fine model %s" %i)
        fine_model = fine_models['fine_model_%s' %i]
        fine_model.to(device)
        fine_model.eval()
        for counter, (images, labels) in enumerate(test_batch_dl):
            imageGPU = images.to(device)
            outputs = torch.Tensor.cpu(fine_model(imageGPU))
            predicts.append(outputs.detach().numpy())   
        predicts = np.vstack(predicts)
        fine_prediction.append(predicts)

0


/home/transmedia-eg/test_env/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


1
2
3
4
7
8
9
10
11
12
13
14
15
16
18
21
23
25
27


In [84]:
#reformat fine_prediction
fine_prediction = np.hstack(fine_prediction)
result = np.multiply(fine_prediction, coarse_prediction)
result = np.argmax(result, axis=1)
result_decoded = [stacked_fine_labels[i] for i in result]
final_acc = accuracy_score(targets, result_decoded) 
print("overall accuracy is %s" %final_acc)

In [85]:
accuracy_score(targets, result_decoded)

0.6788